In [1]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import pandas as pd

def initalize_driver():
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(chrome_options=chrome_options)
    return driver

driver = initalize_driver()
base_url = "https://www.personality-database.com/"

def get_page_contents(driver, url):
    driver.get(url)
    return driver.page_source

def get_category_urls(driver, base_url):
    page = get_page_contents(driver, base_url)
    soup = BeautifulSoup(page, 'html.parser')
    a = soup.find("div", {"id": "categories"})
    a1 = str(a).split("Real Profiles")[-1].split("Fictional Profiles")[0]
    a1 = BeautifulSoup(a1, 'html.parser')
    a1 = a1.find_all("a")
    a1 = list(map(lambda x: re.findall(r'"/(.*?)"', str(x)), a1))
    cats = [base_url + c[0] for c in a1]
    return cats

def get_subcategory_urls(driver, category_url):
    page = get_page_contents(driver, category_url)    
    soup = BeautifulSoup(page)
    a = soup.find_all("div", {"id" : re.compile(re.compile('^subcat-item.*'))})
    a1 = list(map(lambda x: x.find_all("a"), a))
    a1 = list(map(lambda x: re.findall(r'"/(.*?)"', str(x)), a1))    
    sub_cats = [base_url + c[0] for c in a1]    
    return sub_cats

def get_profiles_urls(driver, subcategory_url):
    page = get_page_contents(driver, subcategory_url)    
    soup = BeautifulSoup(page)
    a = soup.find_all("span", {"class" : re.compile(re.compile('^profile-container.*'))})
    a1 = list(map(lambda x: x.find_all("a"), a))
    a1 = list(map(lambda x: re.findall(r'"/(.*?)"', str(x)), a1))    
    profiles = [base_url + c[0] for c in a1]    
    return profiles

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


In [2]:
# categories = get_categories_pages(driver, base_url)

# sub_cats = get_subcategories(driver, categories[1])

In [3]:
# profiles = get_profiles(driver, sub_cats[0])

In [4]:
def make_categories_df(driver, base_url):
    cats = get_category_urls(driver, base_url)
    categories, subcategories = [], []
    for c in cats:
        sub_cats = get_subcategory_urls(driver, c)
        categories += [c]*len(sub_cats)
        subcategories += sub_cats
    df = pd.DataFrame({"category":categories,"subcategory":subcategories})
    return df

# categories_df = make_categories_df(driver, base_url)
# categories_df.to_csv("data/categories.csv", index=False)
categories_df = pd.read_csv("data/categories.csv")
categories_df.head()

,category,subcategory
0,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...
1,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...
2,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...
3,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...
4,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...


In [5]:
def make_profiles_df(driver, categories_df):
    subcategories, profiles = [], []
    for sc in categories_df["subcategory"]:
        profs = get_profiles_urls(driver, sc)
        subcategories += [sc]*len(profs)
        profiles += profs
    df = pd.DataFrame({"subcategory":subcategories, "profile":profiles})
    df = categories_df.merge(df, on="subcategory")
    return df

# profiles_df = make_profiles_df(driver, categories_df)
# profiles_df.to_csv("data/profiles.csv", index=False)
profiles_df = pd.read_csv("data/profiles.csv")
profiles_df.head()

,category,subcategory,profile
0,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/4...
1,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/7...
2,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/7...
3,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/7...
4,https://www.personality-database.com/category/...,https://www.personality-database.com/subcatego...,https://www.personality-database.com/profile/7...


In [13]:
def parse_profile(driver, profile_url):
    print(profile_url)
    page = get_page_contents(driver, profile_url)    
    soup = BeautifulSoup(page)
    print(soup)
    votes = soup.find_all("div", {"class" : re.compile(re.compile('.*vote-breakdown-container.*'))})
    votes = [v.find_all("p") for v in votes]
    labels = ["votes_"+re.findall(r'(?<=\<p\>)(.*)(?=\()', str(v))[0].strip() for v in votes]
    scores = [re.findall(r'(?<=\()(.*)(?=\))', str(v))[0].strip() for v in votes]
    d = dict(zip(labels, scores))
    print(d)
    letters = soup.find_all("div", {"class" : re.compile(re.compile('.*breakdown-item.*'))})
    labels = [l.find("h1", {"class" : re.compile(re.compile('the-letters'))}).contents[0] for l in letters[:8]]
    scores = [l.find("h5").contents[0] for l in letters[:8]]    
    d2 = dict(zip(labels, scores))
    print(d2)
    labels = [l.find("p").contents[0] for l in letters[8:]]
    labels2 = [l.find("h5").contents[0] for l in letters[8:]]
    d3 = dict(zip(labels, labels2))
    print(d3)
#     d4 = {"profile": profile_url}
    return {**d, **d2, **d3}
#     return pd.DataFrame.from_dict(data={**d, **d2, **d3}, orient="index").transpose()

# driver = initalize_driver()  

# ps = []
# for p in profiles_df["profile"][:3]:
#     ps.append(parse_profile(driver, p))

# profiles_full_df = pd.concat(ps,axis=0)
    
# d = parse_profile(driver, profiles_df["profile"][0])

In [14]:
driver.quit()
driver = initalize_driver()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


In [15]:
parse_profile(driver, profiles_df["profile"][0])

https://www.personality-database.com/profile/449/donald-trump-presidents-of-the-usa-mbti-personality-type
<html xmlns="http://www.w3.org/1999/xhtml"><head></head><body></body></html>
{}
{}
{}


{}

In [74]:
s = '"<p>ESTP   (471)</p>, <p class="keirsey">The Promoter</p>"'

In [79]:
re.findall(r'(?<=\<p\>)(.*)(?=\()', s)[0].strip()

'ESTP'